In [0]:
from __future__ import division

import numpy as np
import pandas as pd
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier)
from sklearn.model_selection import cross_validate
from sklearn import metrics, linear_model, preprocessing
from sklearn.model_selection import train_test_split
import logging

In [0]:
# metrics, SEED = 42  # always use a seed for randomized procedures
SEED = 42  # always use a seed for randomized procedures
def save_results(predictions, filename):
    """Given a vector of predictions, save results in CSV format."""
    with open(filename, 'w') as f:
        f.write("id,ACTION\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (i + 1, pred))

In [5]:
"""
Fit models and make predictions.
We'll use one-hot encoding to transform our categorical features
into binary features.
y and X will be numpy array objects.
"""
# === load data in memory === #
print ("loading data")
X = pd.read_csv('D:/applied/amazon/train.csv')
X = X.drop(['ROLE_CODE'], axis=1)
y = X['ACTION']
X = X.drop(['ACTION'], axis=1)
X_test = pd.read_csv('D:/applied/amazon/test.csv', index_col=0)
X_test = X_test.drop(['ROLE_CODE'], axis=1)
X_test['ACTION'] = 0
y_test = X_test['ACTION']
X_test = X_test.drop(['ACTION'], axis=1)

loading data


In [0]:
modelRF =RandomForestClassifier(n_estimators=1999, max_features='sqrt', max_depth=None, min_samples_split=9, random_state=SEED)#8803
modelXT =ExtraTreesClassifier(n_estimators=1999, max_features='sqrt', max_depth=None, min_samples_split=8, random_state=SEED) #8903
modelGB =GradientBoostingClassifier(n_estimators=50, learning_rate=0.20, max_depth=20, min_samples_split=9, random_state=SEED)  #8749
# 599: 20/90/08
#1999: 24/95/06

X_all = pd.concat([X_test,X], ignore_index=True)

# I want to combine role_title as a subset of role_familia and see if same results
X_all['ROLE_TITLE'] = X_all['ROLE_TITLE'] + (1000 * X_all['ROLE_FAMILY'])
X_all['ROLE_ROLLUPS'] = X_all['ROLE_ROLLUP_1'] + (10000 * X_all['ROLE_ROLLUP_2'])
X_all = X_all.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2','ROLE_FAMILY'], axis=1)

In [9]:
# Count/freq
print( "Counts")
for col in X_all.columns:
    X_all['cnt'+col] = 0
    groups = X_all.groupby([col])
    for name, group in groups:
        count = group[col].count()
        X_all['cnt'+col].ix[group.index] = count 
    X_all['cnt'+col] = X_all['cnt'+col].apply(np.log) # could check if this is neccesary, I think probably not

Counts


C:\Users\CHEN\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  import sys
C:\Users\CHEN\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
# Percent of dept that is this resource
for col in X_all.columns[1:6]:
    X_all['Duse'+col] = 0.0
    groups = X_all.groupby([col])
    for name, group in groups:
        grps = group.groupby(['RESOURCE'])
        for rsrc, grp in grps:
            X_all['Duse'+col].ix[grp.index] = float(len(grp.index)) / float(len(group.index) )

C:\Users\CHEN\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  import sys


In [12]:
# Number of resources that a manager manages
for col in X_all.columns[0:1]:
    if col == 'MGR_ID':
        continue
    print (col)
    X_all['Mdeps'+col] = 0
    groups = X_all.groupby(['MGR_ID'])
    for name, group in groups:
        X_all['Mdeps'+col].ix[group.index] = len(group[col].unique()) 

RESOURCE


C:\Users\CHEN\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


In [0]:
X = X_all[:][X_all.index>=len(X_test.index)]
X_test = X_all[:][X_all.index<len(X_test.index)]

# === Combine Models === #
# Do a linear combination using a cross_validated data split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.5, random_state=SEED)

In [22]:
logging.info('start') # 4:28
modelRF.fit(X_cv, y_cv) 
modelXT.fit(X_cv, y_cv) 
modelGB.fit(X_cv, y_cv) 
predsRF = modelRF.predict_proba(X_train)[:, 1]
predsXT = modelXT.predict_proba(X_train)[:, 1]
predsGB = modelGB.predict_proba(X_train)[:, 1]
preds = np.hstack((predsRF, predsXT, predsGB)).reshape(3,len(predsGB)).transpose()
preds[preds>0.9999999]=0.9999999
preds[preds<0.0000001]=0.0000001
preds = -np.log((1-preds)/preds)
modelEN1 = linear_model.LogisticRegression()
modelEN1.fit(preds, y_train)
logging.info('end')
print (modelEN1.coef_)

[[0.22143483 0.98816299 0.06566175]]


C:\Users\CHEN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [24]:
modelRF.fit(X_train, y_train) 
modelXT.fit(X_train, y_train) 
modelGB.fit(X_train, y_train) 
predsRF = modelRF.predict_proba(X_cv)[:, 1]
predsXT = modelXT.predict_proba(X_cv)[:, 1]
predsGB = modelGB.predict_proba(X_cv)[:, 1]
preds = np.hstack((predsRF, predsXT, predsGB)).reshape(3,len(predsGB)).transpose()
preds[preds>0.9999999]=0.9999999
preds[preds<0.0000001]=0.0000001
preds = -np.log((1-preds)/preds)
modelEN2 = linear_model.LogisticRegression()
modelEN2.fit(preds, y_cv)
print (modelEN2.coef_)

[[0.26812988 0.92114096 0.05551378]]


C:\Users\CHEN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [25]:
coefRF = modelEN1.coef_[0][0] + modelEN2.coef_[0][0]
coefXT = modelEN1.coef_[0][1] + modelEN2.coef_[0][1]
coefGB = modelEN1.coef_[0][2] + modelEN2.coef_[0][2]

# === Predictions === #
# When making predictions, retrain the model on the whole training set
modelRF.fit(X, y)
modelXT.fit(X, y)
modelGB.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.2, loss='deviance', max_depth=20,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=9,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
### Combine here
predsRF = modelRF.predict_proba(X_test)[:, 1]
predsXT = modelXT.predict_proba(X_test)[:, 1]
predsGB = modelGB.predict_proba(X_test)[:, 1]
predsRF[predsRF>0.9999999]=0.9999999
predsXT[predsXT>0.9999999]=0.9999999
predsGB[predsGB>0.9999999]=0.9999999
predsRF[predsRF<0.0000001]=0.0000001
predsXT[predsXT<0.0000001]=0.0000001
predsGB[predsGB<0.0000001]=0.0000001
predsRF = -np.log((1-predsRF)/predsRF)
predsXT = -np.log((1-predsXT)/predsXT)
predsGB = -np.log((1-predsGB)/predsGB)
preds = coefRF * predsRF + coefXT * predsXT + coefGB * predsGB

In [0]:
save_results(preds, "D:/applied/amazon/original_jpyternotebook_result/result_1.csv")